In [1]:
import xarray as xr
import numpy as np
import importlib
from matplotlib import pyplot as plt
import DA_core as DA
from glob import glob
import torch.utils.data as Data
from torch import optim
import torch
from torchsummary import summary
import ML_core as ML
from numpy.random import default_rng
import os

rng = default_rng()
DA.read_data_dir='/scratch/cimes/feiyul/PyQG/data/training'
DA.save_data_dir='/scratch/cimes/feiyul/PyQG/data/training'

data_dir='/scratch/cimes/feiyul/PyQG/data'
# data_dir='/net2/fnl/PyQG/data'
B_ens_kws={'cmap':'bwr','levels':np.linspace(-2.5E-11,2.5E-11,26),'extend':'both'}
B_ens_kws1={'cmap':'bwr','levels':np.linspace(-0.25E-11,0.25E-11,26),'extend':'both'}
q_kws={'cmap':'bwr','levels':np.linspace(-3.4E-5,3.4E-5,18),'extend':'both'}

INFO:  Logger initialized


In [6]:
importlib.reload(ML)

<module 'ML_core' from '/home/feiyul/PyQG/pyqg_experiments/examples/DA/ML_core.py'>

In [10]:
os.system('nvidia-smi')

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



2304

In [ ]:
torch.cuda.is_available()

In [2]:
DA_paras={'nens':1280,
          'DA_method':'EnKF',
          'Nx_DA':64,
          'Nx_truth':128,
          'obs_freq':10,
          'obs_err':[1,-5,5,-7],
          'DA_freq':10,
          'save_B':False,
          'nobs':[50,50],
          'R_W':100,
          'inflate':[1,0.45]}
DA_exp=DA.DA_exp(**DA_paras)
print(DA_exp.file_name())
# obs_ds=DA_exp.read_obs()
in_ch=[0,1]
out_ch=[0,1,2]

DA_paras1={'nens':320,
          'DA_method':'EnKF',
          'Nx_DA':64,
          'Nx_truth':128,
          'obs_freq':10,
          'obs_err':[1,-6,2,-8],
          'DA_freq':10,
          'save_B':False,
          'nobs':[100,0],
          'R_W':100,
          'inflate':[1,0.55]}
DA_exp1=DA.DA_exp(**DA_paras1)

EnKF_Nx64_from_Nx128_ens1280_freq10_relax0.45_R100_nobs50_50_err1E-5_5E-7


In [3]:
mean_ds=DA_exp.read_mean().load()
print(mean_ds.q.shape)
# if DA_exp.nens>1:  
#     std_ds=DA_exp.read_std()

# B_ens_ds=xr.open_mfdataset(['{}/{}/B_ens_day{:04d}.nc'.format(data_dir,DA_exp.file_name(),day) for day in np.arange(9,3650,10)])
B_ens_ds=xr.open_dataset('{}/training/{}/B_ens.nc'.format(data_dir,DA_exp.file_name()))
print(B_ens_ds.B_ens.shape)

# ml_std_ds=xr.open_dataset('./ML/{0}/std_{0}.nc'.format(DA_exp.file_name()))
# print(ml_std_ds)

B_R=int((len(B_ens_ds.x_d)-1)/2)
B_size=24
B_start=1

DA_days=slice(369,3650,DA_exp.DA_freq)
DA_it=slice(int((DA_days.start-DA_exp.DA_freq+1)/DA_exp.DA_freq),int((DA_days.stop-DA_exp.DA_freq+1)/DA_exp.DA_freq)+1)
print(DA_days,DA_it)
i_x=slice(0,DA_exp.Nx_DA)
i_y=slice(0,DA_exp.Nx_DA)

/scratch/cimes/feiyul/PyQG/data/training/EnKF/EnsMean_EnKF_Nx64_from_Nx128_ens1280_freq10_relax0.45_R100_nobs50_50_err1E-5_5E-7.nc
(3650, 2, 64, 64)
(365, 2, 64, 64, 2, 27, 27)
slice(369, 3650, 10) slice(36, 365, None)


In [4]:
B_shape=B_ens_ds.B_ens.isel(time=DA_it,y=i_y,x=i_x).shape
print(B_shape)
print(mean_ds.q.isel(time=DA_days).shape)
B_nt=B_shape[0]
B_ny=B_shape[2]
B_nx=B_shape[3]
B_total=B_nt*B_ny*B_nx
print(B_total)
n_train=int(B_total*0.8)
# rngs=rng.permutation(B_total)
rngs=np.arange(B_total)
partition={'train':rngs[0:n_train],'valid':rngs[n_train:]}
        
train_ds=ML.Dataset(mean_ds.q.isel(time=DA_days),DA_exp.Nx_DA,
                    B_ens_ds.B_ens.isel(time=DA_it,y=i_y,x=i_x),
                    partition['train'],ml_std_ds.q_std.data,ml_std_ds.B_std.data,in_ch,out_ch,B_size=B_size,B_start=B_start)
valid_ds=ML.Dataset(mean_ds.q.isel(time=DA_days),DA_exp.Nx_DA,
                    B_ens_ds.B_ens.isel(time=DA_it,y=i_y,x=i_x),
                    partition['valid'],ml_std_ds.q_std.data,ml_std_ds.B_std.data,in_ch,out_ch,B_size=B_size,B_start=B_start)

params = {'batch_size':16,'num_workers':1,'shuffle':True}
training_generator = torch.utils.data.DataLoader(train_ds, **params)
validation_generator = torch.utils.data.DataLoader(valid_ds, **params)

(329, 2, 64, 64, 2, 27, 27)
(329, 2, 64, 64)
1347584


NameError: name 'ml_std_ds' is not defined

In [4]:
B_std=np.empty((2,2))
B_std[0,0]=np.std(B_ens_ds.B_ens.isel(time=DA_it,lev=0,lev_d=0))
B_std[0,1]=np.std(B_ens_ds.B_ens.isel(time=DA_it,lev=0,lev_d=1))
B_std[1,0]=B_std[0,1]
B_std[1,1]=np.std(B_ens_ds.B_ens.isel(time=DA_it,lev=1,lev_d=1))
print(B_std)

q_std=np.zeros((2,1))
q_std[0]=np.std(mean_ds.q.isel(time=DA_days,lev=0))
q_std[1]=np.std(mean_ds.q.isel(time=DA_days,lev=1))
print(q_std)

B_da=xr.DataArray(B_std,coords=[mean_ds.lev,mean_ds.lev])
q_da=xr.DataArray(q_std.squeeze(),coords=[mean_ds.lev])
std_ds=xr.Dataset({'B_std':B_da,'q_std':q_da})
std_ds.to_netcdf('./ML/{0}/std_{0}.nc'.format(DA_exp.file_name()))

[[9.84226755e-12 1.49484862e-13]
 [1.49484862e-13 3.69689238e-15]]
[[6.64853917e-06]
 [1.22305091e-07]]


In [9]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
model=ML.Unet(in_ch=len(in_ch),out_ch=len(out_ch))
model=model.to(device)
print(device)

# check keras-like model summary using torchsummary
summary(model, input_size=(len(in_ch),train_ds.B_size,train_ds.B_size))

cpu
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 16, 16]             304
              ReLU-2           [-1, 16, 16, 16]               0
            Conv2d-3           [-1, 16, 16, 16]           2,320
              ReLU-4           [-1, 16, 16, 16]               0
         MaxPool2d-5             [-1, 16, 8, 8]               0
            Conv2d-6             [-1, 32, 8, 8]           4,640
              ReLU-7             [-1, 32, 8, 8]               0
            Conv2d-8             [-1, 32, 8, 8]           9,248
              ReLU-9             [-1, 32, 8, 8]               0
        MaxPool2d-10             [-1, 32, 4, 4]               0
           Conv2d-11             [-1, 64, 4, 4]          18,496
             ReLU-12             [-1, 64, 4, 4]               0
           Conv2d-13             [-1, 64, 4, 4]          36,928
             ReLU-14             [-

In [10]:
criterion = torch.nn.MSELoss() # MSE loss function
optimizer = optim.Adam(model.parameters(), lr=0.002)

In [11]:
model=model.double()
n_epochs = 30 #Number of epocs
validation_loss = list()
train_loss = list()
start_epoch=0
if start_epoch>0:
    model_file='./ML/unet_epoch{}_in{}_out{}_B{}_{}.pt'.format(start_epoch,''.join(map(str,in_ch)),''.join(map(str,out_ch)),B_size,DA_exp.file_name())
    print(model_file)
    model.load_state_dict(torch.load(model_file,map_location=torch.device('cpu')))
# time0 = time()  
for epoch in range(start_epoch+1, n_epochs + 1):
    train_loss.append(ML.train_model(model,criterion,training_generator,optimizer,device))
    # validation_loss.append(ML.test_model(model,criterion,validation_generator,optimizer,device))
    torch.save(model.state_dict(), './ML/{}/unet_epoch{}_in{}_out{}_B{}_{}.pt'.\
        format(DA_exp.file_name(),epoch,''.join(map(str,in_ch)),''.join(map(str,out_ch)),B_size,DA_exp.file_name()))

KeyboardInterrupt: 

In [ ]:
plt.plot(train_loss,'b', label='training loss');
plt.plot(validation_loss,'r', label='validation loss');

plt.legend();